# create

> Fill in a module description here

In [ ]:
#| default_exp envs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, pathlib, json



In [ ]:
import localenv

In [ ]:
#| export

def _create_getenv(env_vars:dict, def_getenv=os.getenv):
    '''Internal function to create a getenv function using the dict specified
    environment variabels and a fallback getenv function'''

    def _getenv(V, D=None):
        '''Locally created with user dictionary'''
        assert V is not None, f"Can not have '{str(V)}' as var name"
        return env_vars.get(V, def_getenv(V,D))

    return _getenv


In [ ]:
pathlib.Path.cwd().joinpath('.env.json').exists()

False

In [ ]:
current_path = pathlib.Path.cwd()
home = pathlib.Path.home()
root = pathlib.Path(current_path.root)


In [ ]:
current_path.relative_to(root)

Path('home/fenke/repos/localenv/nbs')

In [ ]:
root.relative_to(root)

Path('.')

In [ ]:
# invariant:current_path = current_path.joinpath(home, current_path.relative_to(home))
try:
    while not current_path.joinpath(home, current_path.relative_to(home), '.env-f.json').exists():
        print(current_path, current_path.joinpath(home, current_path.relative_to(home), '.env.json'), current_path.joinpath(home, current_path.relative_to(home), '.env-f.json').exists())
        current_path = current_path.parent

    print(current_path)

except ValueError:
    print(home)
    



/home/fenke/repos/localenv/nbs /home/fenke/repos/localenv/nbs/.env.json False
/home/fenke/repos/localenv /home/fenke/repos/localenv/.env.json False
/home/fenke/repos /home/fenke/repos/.env.json False
/home/fenke /home/fenke/.env.json False
/home/fenke


In [ ]:
current_path = pathlib.Path.cwd()
home = pathlib.Path.home()
root = pathlib.Path(home.root)

In [ ]:
list(current_path.parents)

[Path('/home/fenke/repos/localenv'),
 Path('/home/fenke/repos'),
 Path('/home/fenke'),
 Path('/home'),
 Path('/')]

In [ ]:
for p in current_path.parents:
    if p.joinpath('.env.json').exists():
        print(p)
        break

/home/fenke/repos


In [ ]:
#| export
def find_env_path(env_filename = '.env.json'):

    current_path = pathlib.Path.cwd()
    for p in current_path.parents:
        if p.joinpath(env_filename).exists():
            return p.joinpath(env_filename)

    return pathlib.Path.home().joinpath(env_filename)



In [ ]:
find_env_path(".env.json")

Path('/home/fenke/repos/.env.json')

In [ ]:
env_filename = ".env.json"
env_path = find_env_path(env_filename)
print(env_path)


/home/fenke/repos/.env.json


In [ ]:

with open(env_path) as f:
    env_vars = json.load(f)

list(env_vars.keys())
list(env_vars['production'].keys())

['USR_TOKEN', 'API_URL', 'API_TOKEN', 'VC_API', 'VC_TOKEN', 'bar']

In [ ]:
#| export
#| eval: false

with open(find_env_path()) as f:
    env_vars = json.load(f)

for k, e in env_vars.items():
    globals()[k] = _create_getenv(env_vars=e)

#globals()['__all__'] = list(set(env_vars.keys()) | set(globals().get('__all__',[])))


In [ ]:
from localenv.envs import production as getenv

In [ ]:
getenv('bar')

'baz'

In [ ]:
os.getenv('bar')

## Additional supportive functions

In [ ]:
#| export

def get_var_from_env_file(var_name, default_getenv=os.getenv, env_filename = '.env.json'):
    env_path = find_env_path(env_filename)
    with open(env_path) as f:
        env_vars = json.load(f)
    return env_vars.get(var_name, default_getenv(var_name))

In [ ]:
get_var_from_env_file("foo")

{'bar': 'baz'}

In [ ]:
get_var_from_env_file(("HOME"))

'/home/fenke'

In [ ]:
#| export

def update_env_file(section, var_name, var_value, env_filename = '.env.json'):
    env_path = find_env_path(env_filename)
    with open(env_path) as f:
        env_vars = json.load(f)

    section_vars = env_vars.get(section, {})
    section_vars[var_name] = var_value
    env_vars[section] = section_vars

    with open(env_path, 'w') as f:
        json.dump(env_vars, f, indent=3, ensure_ascii=True)

In [ ]:
update_env_file("production", "bar", "baz")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()